# Internal Coordinates and the B-matrix

## Defining bonds
Assuming a reasonable initial geometrical input, the identification of which atoms are bonded may, in most cases, be simply achieved using standard covalent radii.

In [1]:
import psi4
import numpy as np
mol = psi4.geometry("""
 1 1
  pubchem:ammonium
""")
mol.update_geometry()
xyzGeom = np.array(mol.geometry())
print ("Starting Geometry for Ammonium")
print (xyzGeom) 

# Determine the connectivity using standard covalent radii
from optking import covRadii, v3d
from itertools import combinations, permutations
from math import sqrt
Natom = len(xyzGeom)
Z= [mol.Z(i) for i in range(Natom)]
C = np.zeros( (Natom, Natom), bool )
for i,j in combinations( range(Natom), 2):
  R = v3d.dist( xyzGeom[i], xyzGeom[j])
  Rcov = (covRadii.R[int(Z[i])] + covRadii.R[int(Z[j])])/0.52917720859
  if R < 1.3 * Rcov:
    C[i,j] = C[j,i] = True
    
print (C)


	Searching PubChem database for ammonium
	Found 1 result
Starting Geometry for Ammonium
[[  0.00000000e+00  -8.29781462e-35   3.92826956e-06]
 [  1.58424594e+00   4.85035429e-17   1.12021175e+00]
 [  4.85021276e-17  -1.58419971e+00  -1.12023904e+00]
 [ -1.58424594e+00  -4.85035429e-17   1.12021175e+00]
 [ -4.85021276e-17   1.58419971e+00  -1.12023904e+00]]
[[False  True  True  True  True]
 [ True False False False False]
 [ True False False False False]
 [ True False False False False]
 [ True False False False False]]


## Determining internal coordinates automatically
Once the connectivity is determined, the chemically relevant bonds, angles, and dihedral angles can be determined from the connected atoms.  A stretching coordinate is added for every two bonded atoms, and a bending coordinate is added for all A-B-C sequences of connected atoms. Linear bends require special treatment that will not be shown here.  For this example of ammonia, there are no dihedral angles.  However, generally any bonded sequences A-B-C-D are also taken to define a dihedral coordinate that is added to the set of optimization coordinates.

In [2]:
intcos = []
# add stretches by connectivity
from optking import stre
for i,j in combinations( range(Natom), 2 ):
  if C[i,j]:
    s = stre.STRE(i, j)
    if s not in intcos:
      intcos.append(s)

# add bends by connectivity
from optking import bend
for i,j in permutations( range(Natom), 2):
  if C[i,j]:
    for k in range(i+1,Natom):  # make i<k
      if C[j,k]:
        (check, val) = v3d.angle(xyzGeom[i], xyzGeom[j], xyzGeom[k])
        if not check: continue
        b = bend.BEND(i, j, k)
        if b not in intcos:
          intcos.append(b)
          
# print our internal coordinate set with values
for i in intcos:
    print "%10s = %10.5f" % (i, i.q(xyzGeom))

    R(1,2) =    1.94028
    R(1,3) =    1.94027
    R(1,4) =    1.94028
    R(1,5) =    1.94027
  B(2,1,3) =    1.91064
  B(2,1,4) =    1.91065
  B(2,1,5) =    1.91064
  B(3,1,4) =    1.91064
  B(3,1,5) =    1.91060
  B(4,1,5) =    1.91064


## Redundant Coordinates
These simple internal coordinates have many advantages. They tend to produce a strongly diagonal energy second derivative matrix (Hessian), and they have force constants which are readily guessable, i.e., transferable from one molecule to another. In these two respects, they are vastly superior to Cartesian coordinates.

For many years, the primary difficulty is that the simple procedure demonstrated here generates too many coordinates. Non-linear molecules have 3N-6 internal degrees of freedom (where N is the number of atoms), but including all the internal coordinates defined by the bond connectivity will generally produce more than that.

For ammonia, we see that we have 10 coordinates, while the ion has only 9 internal degrees of freedom. Various alternative coordinate systems were explored, for example "delocalized coordinates" [J.Baker, A. Kessi, and B. Delley, _J. Chem. Phys._, 105, 192 (1996)], and "natural internal coordinates" [G. Fogarasi, X. Zhou, P.W. Taylor, and P. Pulay, _J. Am. Chem. Soc._, 114, 8191 (1992)].  However, an elegant and robust solution turned out to be to simply work with a redundant set and to make the necessary mathematical adjustments.  This approach was advanced by P. Pulay and G. Fogarasi [_J. Chem. Phys._, 96, 2856 (1992)].

## The B matrix and coordinate transformations

The Wilson B-matrix [see the classic text E.B. Wilson, J.C. Decius, and P.C. Cross, _Molecular Vibrations_, Dover (1955)] is the key to typical transformations between internal and Cartesian coordinates.  A B-matrix element is defined as the derivative of an internal coordinate value with respect to a Cartesian coordinate.

$$\textbf {B}_{ij} = \frac{\delta q_i}{\delta x_j}$$

The B-matrix therefore has dimensions of (Number of internals) by (Number of Cartesians, or 3N).  A row of the B-matrix (corresponding to a particular internal coordinate) may also be interpreted as the set of Cartesian unit vectors for each atom along which displacing would result in a maximal increase in the value of the coordinate.  (These unit vectors are sometimes called "s-vectors".)  Formulas for these B-matrix components as well as their derivatives are provided in a 2002 paper [V. Bakken and T. Helgaker, _J. Chem. Phys._, 117, 9160 (2002)], albeit not without some errors in the dihedral derivative B-matrix formulas.

Lets take a look at the first row of the B-matrix for ammonia:

In [3]:
from optking import intcosMisc
Bmat = intcosMisc.Bmat(intcos, xyzGeom)
print('B matrix - first row')
print(Bmat[0])

B matrix - first row
[ -8.16502224e-01  -2.49981709e-17  -5.77342289e-01   8.16502224e-01
   2.49981709e-17   5.77342289e-01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]


Notice that only the first 6 elements corresponding to the Cartesian coordinates
of the first two atoms are non-zero. Since the first internal is R(1,2), only the first 2 atoms are relevant to its definition.  The B-matrix, therefore, is tremendously sparse, when the internal coordinates are restricted to stretches, bends, and dihedrals.

For a diatomic, the B-matrix consists of a single row, and the components are simply two vectors pointing outward as shown below.

In [4]:
H2mol = psi4.geometry("""
H
H 1 0.8
""")
H2intcos = [ stre.STRE(0,1) ]
H2mol.update_geometry()
H2xyzGeom = np.array(mol.geometry())
H2Bmat = intcosMisc.Bmat(H2intcos, H2xyzGeom)
print("B matrix for hydrogen molecule:")
print(H2Bmat)


B matrix for hydrogen molecule:
[[ -8.16502224e-01  -2.49981709e-17  -5.77342289e-01   8.16502224e-01
    2.49981709e-17   5.77342289e-01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00]]


The B-matrix is immediately useful for transforming small displacements in Cartesian coordinates into internal coordinates.

$$\textbf {B}_{ij} \delta x_j = \delta q_i$$

or in matrix vector form

$$\textbf {B} \Delta \textbf {x} = \Delta \textbf {q} $$

However, since the B-matrix depends upon the geometry, this transformation is exact only for infinitesimal displacements.

## Transformations between internal and Cartesian forces
The B-matrix readily facilitates the transformation of the energy gradient into Cartesian coordinates from internal coordinates.

$$ \frac{\delta E}{\delta q_i} \frac{\delta q_i}{\delta x_j} = \frac{\delta E}{\delta x_j}$$ or $$ \textbf{B}^T \textbf{g}_q = \textbf{g}_x$$ 

The code below shows how a Cartesian gradient computed with Psi4 for ammonia is then transformed into a gradient in internal coordinates.

The reverse transformation is of greater practical importance. Quantum chemistry codes typically provide an energy gradient in Cartesian coordinates.  This gradient must be transformed into internal coordinates, in preparation for whatever the step-determining algorithm (Newton-Raphson, etc.) will be used. This transformation is more complicated because 1) we need to invert the non-square B-matrix; and 2) the B-matrix is often redundant, i.e., the rows are linearly dependent.

The left, generalized inverse of $\textbf {B}^T$, we will call the $\textbf A$ matrix.  $\textbf A$ has dimension (Num. of Cartesians) by (Num. of internals) and is defined by

$$ \textbf A = (\textbf{B} \textbf{u} \textbf {B}^T)^{-1} \textbf {B} \textbf{u}$$

where u is an arbitrary matrix of dimension (Num. of Cartesians) by (Num. of Cartesians).

That $\textbf A$ is the left inverse of $\textbf {B}^T$ may be readily seen by the following:

$$ \textbf{B}^T \textbf{g}_q = \textbf{g}_x$$
$$ \textbf{A} \textbf{B}^T \textbf{g}_q = \textbf{A} \textbf{g}_x$$

$$ [ (\textbf{B} \textbf{u} \textbf{B}^T )^{-1} \textbf{B} \textbf{u} ] \textbf{B}^T \textbf{g}_q = \textbf{A} \textbf{g}_x $$
$$(\textbf{B} \textbf{u} \textbf{B}^T )^{-1} ( \textbf{B} \textbf{u} \textbf{B}^T ) \textbf{g}_q = \textbf{A} \textbf{g}_x $$

$$ \textbf{g}_q  = (\textbf{B} \textbf{u} \textbf{B}^T )^{-1} \textbf{B} \textbf{u} \textbf{g}_x $$





The first step in geometry optimization after the internal coordinates have been determined and a Cartesian energy derivative has been computed is to use this expression to compute the gradient in internal coordinates.  (One can instead think in terms of "forces", the only difference is a minus sign.)  

For transforming energy derivates, the choice of $\textbf{u}$ is irrelevant.  In some spectroscopic applications, it is comprised of inverse of atomic masses on its diagonal.  For a thorough exposition on geometric derivative transformaions see the paper by W.D. Allen and A.G. Csaszar [_J. Chem. Phys._, 98, 2983 (1993)].

If $\textbf{u}$ is chosen to be the unit matrix, then the transformation reduces to

$$ \textbf{g}_q  = (\textbf{B} \textbf{B}^T )^{-1} \textbf{B} \textbf{g}_x $$

The matrix $\textbf{B} \textbf{B}^T$ will be symmetric.  If redundant coordinates are present, then the generalized inverse (or pseudo-inverse) must be computed.  The matrix is diagonalized, then the non-zero eigenvalues on the diagonal are inverted, and the matrix is transformed back.  The 


Below is an example of the transformation of the gradient from Cartesians to internals for ammonia.

In [30]:
mol = psi4.geometry("""
1 1
  pubchem:ammonium
""")
mol.update_geometry()
psi4gradientMatrix = psi4.gradient('scf')
gx = np.reshape( np.array(psi4gradientMatrix), (3*Natom))
# Removing some noise
from math import fabs
gx[:] = [0 if fabs(x) < 1.0e-16 else x for x in gx]
print ("Gradient in Cartesian coordinates")
print ( gx )

# Function to return a generalized inverse
def symmMatInv(A):
    dim = A.shape[0]
    det = 1.0

    evals, evects = np.linalg.eigh(A)
    evects = evects.T
    for i in range(dim):
        det *= evals[i]

    diagInv = np.zeros( (dim,dim), float)
    for i in range(dim):
        if fabs(evals[i]) > 1.0e-10:
            diagInv[i,i] = 1.0/evals[i]
            
    # A^-1 = P^t D^-1 P
    tmpMat = np.dot(diagInv, evects)
    AInv = np.dot(evects.T, tmpMat)
    return AInv

B = intcosMisc.Bmat(intcos, xyzGeom)
temp_arr = np.dot(B,gx.T)
G    = np.dot(B, B.T)
Ginv = symmMatInv(G)
gq = np.dot(Ginv,temp_arr.T)

print ("Gradient in Internal coordinates.")
print ( gq )

	Searching PubChem database for ammonium
	Found 1 result
Gradient in Cartesian coordinates
[  0.00000000e+00   0.00000000e+00  -2.06186441e-06   8.05561725e-03
   0.00000000e+00   5.69463262e-03   0.00000000e+00  -8.04588669e-03
  -5.69360169e-03  -8.05561725e-03   0.00000000e+00   5.69463262e-03
   0.00000000e+00   8.04588669e-03  -5.69360169e-03]
Gradient in Internal coordinates.
[  9.86518163e-03   9.85664151e-03   9.86518163e-03   9.85664151e-03
   6.97847154e-07   2.96470698e-06   6.97847154e-07   6.97847154e-07
  -5.75634633e-06   6.97847154e-07]


Lets try the reverse transformation and see if we return to the original gradient.

In [31]:
gx2 = np.dot(B.T,gq)
print ("Gradient in Cartesian coordinates (from internals).")
# Removing some noise
gx2[:] = [0 if fabs(x) < 1.0e-16 else x for x in gx2]
print ( gx2 )


Gradient in Cartesian coordinates (from internals).
[  0.00000000e+00   0.00000000e+00  -2.06186441e-06   8.05561725e-03
   0.00000000e+00   5.69463262e-03   0.00000000e+00  -8.04588669e-03
  -5.69360169e-03  -8.05561725e-03   0.00000000e+00   5.69463262e-03
   0.00000000e+00   8.04588669e-03  -5.69360169e-03]


For information on the transformations of energy second derivatives, see the separate tutorial file on Hessians.